# Library

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import FastText
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','direktor']])
df.head()

,detokenize,direktor
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,0
2,plot utama orang deserter pursuit buru wamil j...,0
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,0


In [3]:
reviews = df['detokenize']
label = df['direktor']

# Variables

In [4]:
MAX_FEATURES = 1000
test_size_split = 0.2

# Feature Extraction

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = MAX_FEATURES)

tfidf = vectorizer.fit_transform(reviews.astype(str))

# SMOTE

In [6]:
smote = SMOTE()

x_smote, y_smote = smote.fit_resample(tfidf, label)

# Split Data

In [7]:
x_train, x_test, y_train, y_test = (
    train_test_split(x_smote, 
                     y_smote, 
                     test_size=test_size_split, 
                     stratify = y_smote, 
                     random_state=42)
)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (35407, 1000)	Y shape: (35407,)
Test	| X shape: (8852, 1000)	Y shape: (8852,)


# Feature Expansion

In [8]:
# Load trained fasttext model
model_exp = FastText.load('fasttext_news_tweet.model').wv

In [9]:
def feature_expansion(df, feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 10) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [10]:
feature_ext = vectorizer.get_feature_names_out()

df_x_train = pd.DataFrame(x_train.todense(), columns = feature_ext)
df_x_test = pd.DataFrame(x_test.todense(), columns = feature_ext)

df_x_train_exp = feature_expansion(df_x_train, feature_ext)
df_x_test_exp = feature_expansion(df_x_test, feature_ext)

x_train = np.expand_dims(df_x_train_exp, 1)
x_test = np.expand_dims(df_x_test_exp, 1)

y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 58.49it/s]


# Model RNN

In [11]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    rnn.build(x_train_c.shape)
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_tfidf_ft_smote_direktor.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_tfidf_ft_smote_direktor.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [12]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
274/277 [============================>.] - ETA: 0s - loss: 0.5235 - accuracy: 0.7851
Epoch 1: val_accuracy improved from -inf to 0.91426, saving model to rnn_tfidf_ft_smote_direktor.h5
277/277 [==============================] - 10s 13ms/step - loss: 0.5211 - accuracy: 0.7862 - val_loss: 0.2289 - val_accuracy: 0.9143
Epoch 2/10
277/277 [==============================] - ETA: 0s - loss: 0.1700 - accuracy: 0.9385
Epoch 2: val_accuracy improved from 0.91426 to 0.95142, saving model to rnn_tfidf_ft_smote_direktor.h5
277/277 [==============================] - 3s 13ms/step - loss: 0.1700 - accuracy: 0.9385 - val_loss: 0.1332 - val_accuracy: 0.9514
Epoch 3/10
274/277 [============================>.] - ETA: 0s - loss: 0.0795 - accuracy: 0.9731
Epoch 3: val_accuracy improved from 0.95142 to 0.96826, saving model to rnn_tfidf_ft_smote_direktor.h5
277/277 [==============================] - 4s 16ms/step - loss: 0.0792 - accuracy: 0.9732 - val_loss: 0.0904 - val_accuracy: 0.9683
Epoch 4/1

277/277 [==============================] - 3s 10ms/step - loss: 0.0232 - accuracy: 0.9929 - val_loss: 0.0911 - val_accuracy: 0.9738
Epoch 7: early stopping
277/277 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2950
           1       0.99      0.93      0.96      2951
           2       0.96      0.99      0.97      2951

    accuracy                           0.97      8852
   macro avg       0.97      0.97      0.97      8852
weighted avg       0.97      0.97      0.97      8852

F1 Score :  0.9736113020173308
Precision :  0.9741716508603954
Recall :  0.9737941174781315
Accuracy :  0.973791233619521
Epoch 1/10
273/277 [============================>.] - ETA: 0s - loss: 0.5115 - accuracy: 0.7875
Epoch 1: val_accuracy improved from -inf to 0.91448, saving model to rnn_tfidf_ft_smote_direktor.h5
277/277 [==============================] - 7s 12ms/step - loss: 0.5081 - accuracy: 0.7891 

# Save Output

In [13]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_tfidf_ft_smote_direktor.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.977103,0.977148,0.977183,0.977180,"[[2947, 3, 0], [54, 2819, 78], [0, 67, 2884]]"
1,2,0.977646,0.978014,0.977748,0.977745,"[[2949, 1, 0], [41, 2790, 120], [1, 34, 2916]]"
2,3,0.973611,0.974172,0.973794,0.973791,"[[2948, 2, 0], [72, 2755, 124], [1, 33, 2917]]"
3,4,0.978802,0.978888,0.978877,0.978875,"[[2946, 4, 0], [48, 2820, 83], [0, 52, 2899]]"
4,5,0.980048,0.980234,0.980120,0.980117,"[[2947, 3, 0], [37, 2816, 98], [0, 38, 2913]]"


In [14]:
print("Accuracy : %.2f" % (df['accuracy'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 97.75
F1-Score : 97.74
